In [ ]:
pip install transformers kobert-transformers

In [ ]:
# 최종
import requests
from bs4 import BeautifulSoup
import json
import torch
from kobert_transformers import get_kobert_model
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 뉴스 기사 URL 입력
url = input("뉴스 url을 입력해주세요: ")

# 웹 페이지 요청
response = requests.get(url)
if response.status_code == 200:
    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 기사 제목 크롤링
    title_tag = soup.find('h2', {'class': 'media_end_head_headline'})
    if title_tag:
        title = title_tag.get_text()
    else:
        title = "Title not found"

    # 기사 본문 크롤링
    content_tag = soup.find('div', {'id': 'newsct_article'})
    if content_tag:
        content = content_tag.get_text(strip=True)
    else:
        content = "Content not found"

    # KoBERT 모델과 토크나이저 로드
    model = BertModel.from_pretrained('monologg/kobert')
    tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

    # 텍스트 요약 함수
    def summarize_text(text, max_length=5):
        sentences = text.split('. ')
        inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        sentence_embeddings = outputs.last_hidden_state.mean(dim=1)

        # 문서 임베딩
        doc_embedding = sentence_embeddings.mean(dim=0).unsqueeze(0)
        similarities = cosine_similarity(doc_embedding, sentence_embeddings)
        top_indices = similarities.argsort()[0][-max_length:]

        summary = [sentences[i] for i in top_indices]
        return ' '.join(summary)

    # 기사 요약
    summary = summarize_text(content, max_length=3)
    print("Summary:", summary)

    # 요약 내용을 로그로 출력하여 확인
    print("요약된 내용:", summary)

    # 카카오 API를 사용하여 요약 내용 전송
    def send_kakao_message(access_token, text):
        url = 'https://kapi.kakao.com/v2/api/talk/memo/default/send'
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
        data = {
            'template_object': json.dumps({
                'object_type': 'text',
                'text': text,
                'link': {
                    'web_url': 'https://developers.kakao.com',
                    'mobile_web_url': 'https://developers.kakao.com'
                }
            })
        }

        # 전송할 내용을 로그로 출력하여 확인
        print("전송할 내용:", text)

        response = requests.post(url, headers=headers, data=data)
        if response.status_code == 200:
            print("Message sent successfully")
        else:
            print(f"Failed to send message: {response.json()}")

    # 발급받은 액세스 토큰 사용
    access_token = '발급받은 토큰입력'
    send_kakao_message(access_token, summary)  # 요약된 내용을 전송합니다.
else:
    print("Failed to retrieve the webpage")